In [1]:
import os
os.chdir("../")
os.getcwd()

'/Users/abishekkumaryadav/DataScience/computer_vision/BoneFracture_classification'

In [64]:
from dataclasses import dataclass
from pathlib import Path

In [65]:
@dataclass(frozen=True)
class ModelTrainingConfig: 
    root_dir: Path
    trained_model_path: Path
    augumentation: Path
    image_size: list
    epoch: int
    batch_size: int
    train_data_path: Path
    valid_data_path: Path
    final_model_path: Path


In [66]:
from src.bone_classifier.constants import *
from src.bone_classifier.utils.common import read_yaml, create_directories
from src.bone_classifier import log

In [67]:
class configurationManager: 
    def __init__(self, config_file_path=CONFIG_FILE_PATH, 
                 params_file_path=PARAMS_FILE_PATH) -> None:
        self.config=read_yaml(config_file_path)
        self.parmas=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_training_config(self)-> ModelTrainingConfig: 
        config=self.config.model_training
        params=self.parmas
        create_directories([config.root_dir])

        model_training_config=ModelTrainingConfig(
            root_dir=config.root_dir,
            augumentation=params.AUGMENTATION,
            trained_model_path=config.trined_model_path,
            image_size=params.IMAGE_SIZE,
            batch_size=params.BATCH_SIZE,
            epoch=params.EPOCHS,
            train_data_path=config.train_data_path,
            valid_data_path=config.valid_data_path,
            final_model_path=config.final_model_path

        )
        return model_training_config

In [68]:
import keras
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

In [69]:
class ModelTraining: 
    def __init__(self, config: ModelTrainingConfig) -> None:
        self.config=config
    

    def get_base_model(self): 
        self.model=keras.models.load_model(self.config.trained_model_path)
    

    def train_valid_generator(self):

        train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)
        validation_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)

        self.train_generator= train_datagen.flow_from_directory(
            self.config.train_data_path,
            target_size=(224,224),
            batch_size=self.config.batch_size
            
        )
        
        self.valid_generator=validation_datagen.flow_from_directory(
            self.config.valid_data_path,
            target_size=(224,224),
            batch_size=self.config.batch_size
        )
    @staticmethod
    def save_model(path: Path, model: keras.models):
        model.save(path)

    

    def train(self):


        self.step_per_epoch=self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps=self.valid_generator.samples // self.valid_generator.batch_size


        
        self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

        self.model.fit(self.train_generator, 

                       epochs=self.config.epoch,
                       steps_per_epoch=self.step_per_epoch,
                       validation_data=self.valid_generator,
                       validation_steps=self.validation_steps)
        
        self.save_model(path=self.config.final_model_path,
                         model=self.model)

    

In [70]:
try: 
    config=configurationManager()
    model_training_config=config.get_model_training_config()
    model_training=ModelTraining(model_training_config)
    model_training.get_base_model()
    model_training.train_valid_generator()
    model_training.train()
except Exception as e: 
    raise e 

[2024-03-16 01:43:41,866 : INFO : common : yaml file : config/config.yaml loaded sucessfully]
[2024-03-16 01:43:41,868 : INFO : common : yaml file : params.yaml loaded sucessfully]
[2024-03-16 01:43:41,868 : INFO : common : Created directories at : artifacts]
[2024-03-16 01:43:41,869 : INFO : common : Created directories at : artifacts/training]
Found 587 images belonging to 5 classes.
Found 73 images belonging to 5 classes.
36/36 [==============================] - 13s 330ms/step - loss: 1.3969 - accuracy: 0.5377 - val_loss: 5.1723 - val_accuracy: 0.2969


/Users/abishekkumaryadav/DataScience/computer_vision/BoneFracture_classification/bone_env/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
